
# Pós tech Fiap - Inteligência Artificial e para devs
## Tech Challenge IADT - Fase 1
### Grupo
- Hiago Marques Rubio - RM
- Mylena Ferreira Lacerda - RM

## Diagnóstico de diabetes
### Data set utilizado [Diabetes Health Indicators Dataset](https://www.kaggle.com/datasets/mohankrishnathalla/diabetes-health-indicators-dataset)


In [1]:
# Import das libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import SelectKBest, f_classif
import warnings
warnings.filterwarnings('ignore')


# Análise exploratória dos dados

## Visão geral do dataset
O dataset contém **100.000 registros** com **31 variáveis** abrangendo diferentes aspectos da saúde e estilo de vida dos pacientes. A análise inicial revela a presença de múltiplos tipos de dados que necessitam de tratamento específico para modelagem de machine learning. Abaixo uma descrição dos dados, e logo após podemos ver as primeiras analises.

## Categorização das variáveis por tipo:

### Variáveis Categóricas (strings - object):
- **Demográficas**: `gender`, `ethnicity`
- **Socioeconômicas**: `education_level`, `income_level`, `employment_status`
- **Comportamentais**: `smoking_status`
- **Clínica**: `diabetes_stage`

### Variáveis Numéricas Discretas (int64):
- **Demográficas**: `age`
- **Estilo de vida**: `alcohol_consumption_per_week`, `physical_activity_minutes_per_week`
- **Histórico médico**: `family_history_diabetes`, `hypertension_history`, `cardiovascular_history`
- **Sinais vitais**: `systolic_bp`, `diastolic_bp`, `heart_rate`
- **Exames laboratoriais**: `cholesterol_total`, `hdl_cholesterol`, `ldl_cholesterol`, `triglycerides`, `glucose_fasting`, `glucose_postprandial`
- **Variável-alvo**: `diagnosed_diabetes`

### Variáveis Numéricas Contínuas (float64):
- **Estilo de vida**: `diet_score`, `sleep_hours_per_day`, `screen_time_hours_per_day`
- **Antropométricas**: `bmi`, `waist_to_hip_ratio`
- **Exames laboratoriais**: `insulin_level`, `hba1c`
- **Score de risco**: `diabetes_risk_score`

## Necessidades de pré-processamento identificadas:

1. **Encoding de variáveis categóricas**: As 7 variáveis categóricas precisarão ser convertidas para formato numérico (One-Hot Encoding, Label Encoding, ou Target Encoding)

2. **Normalização/Padronização**: Variáveis numéricas possuem escalas muito diferentes (ex: idade vs. minutos de atividade física), necessitando normalização

3. **Análise de valores ausentes**: Verificar presença de NaN, especialmente nas variáveis float64

4. **Detecção de outliers**: Principalmente em variáveis clínicas e antropométricas

5. **Análise de correlação**: Identificar multicolinearidade entre variáveis clínicas relacionadas

## Variáveis-alvo para modelagem:
- **`diagnosed_diabetes`** (0/1): Classificação binária supervisionada
- **`diabetes_stage`** (5 categorias): Classificação multiclasse supervisionada  
- **`diabetes_risk_score`** (contínua): Regressão ou clustering não supervisionado  
    

In [2]:
df = pd.read_csv('diabetes_dataset.csv')
print("Número total de linhas no dataset:", len(df))
print("Formato do dataset (linhas, colunas):", df.shape)
print("Número de linhas usando shape[0]:", df.shape[0])
print("\nAs 5 primeiras linhas do dataset:")
df.head()

Número total de linhas no dataset: 100000
Formato do dataset (linhas, colunas): (100000, 31)
Número de linhas usando shape[0]: 100000

As 5 primeiras linhas do dataset:


,age,gender,ethnicity,education_level,income_level,employment_status,smoking_status,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,...,hdl_cholesterol,ldl_cholesterol,triglycerides,glucose_fasting,glucose_postprandial,insulin_level,hba1c,diabetes_risk_score,diabetes_stage,diagnosed_diabetes
0,58,Male,Asian,Highschool,Lower-Middle,Employed,Never,0,215,5.7,...,41,160,145,136,236,6.36,8.18,29.6,Type 2,1
1,48,Female,White,Highschool,Middle,Employed,Former,1,143,6.7,...,55,50,30,93,150,2.00,5.63,23.0,No Diabetes,0
2,60,Male,Hispanic,Highschool,Middle,Unemployed,Never,1,57,6.4,...,66,99,36,118,195,5.07,7.51,44.7,Type 2,1
3,74,Female,Black,Highschool,Low,Retired,Never,0,49,3.4,...,50,79,140,139,253,5.28,9.03,38.2,Type 2,1
4,46,Male,White,Graduate,Middle,Retired,Never,1,109,7.2,...,52,125,160,137,184,12.74,7.20,23.5,Type 2,1


In [3]:
# Verificar informações básicas do dataset
print("Formato do dataset:", df.shape)
print("\nNomes das colunas:")
print(df.columns.tolist())
print("\nTipos de dados:")
print(df.dtypes)

Formato do dataset: (100000, 31)

Nomes das colunas:
['age', 'gender', 'ethnicity', 'education_level', 'income_level', 'employment_status', 'smoking_status', 'alcohol_consumption_per_week', 'physical_activity_minutes_per_week', 'diet_score', 'sleep_hours_per_day', 'screen_time_hours_per_day', 'family_history_diabetes', 'hypertension_history', 'cardiovascular_history', 'bmi', 'waist_to_hip_ratio', 'systolic_bp', 'diastolic_bp', 'heart_rate', 'cholesterol_total', 'hdl_cholesterol', 'ldl_cholesterol', 'triglycerides', 'glucose_fasting', 'glucose_postprandial', 'insulin_level', 'hba1c', 'diabetes_risk_score', 'diabetes_stage', 'diagnosed_diabetes']

Tipos de dados:
age                                     int64
gender                                 object
ethnicity                              object
education_level                        object
income_level                           object
employment_status                      object
smoking_status                         object
alcohol_

In [4]:
# Verificar valores únicos das principais variáveis categóricas
print("Valores únicos - diagnosed_diabetes:", df['diagnosed_diabetes'].unique())
print("Valores únicos - diabetes_stage:", df['diabetes_stage'].unique())
print("Valores únicos - gender:", df['gender'].unique())
print("Valores únicos - smoking_status:", df['smoking_status'].unique())
print("\nResumo estatístico do diabetes_risk_score:")
print(df['diabetes_risk_score'].describe())

Valores únicos - diagnosed_diabetes: [1 0]
Valores únicos - diabetes_stage: ['Type 2' 'No Diabetes' 'Pre-Diabetes' 'Gestational' 'Type 1']
Valores únicos - gender: ['Male' 'Female' 'Other']
Valores únicos - smoking_status: ['Never' 'Former' 'Current']

Resumo estatístico do diabetes_risk_score:
count    100000.000000
mean         30.222362
std           9.061505
min           2.700000
25%          23.800000
50%          29.000000
75%          35.600000
max          67.200000
Name: diabetes_risk_score, dtype: float64


In [5]:
missing_values = df.isnull().sum()
print("Colunas com valores ausentes:")
print(missing_values[missing_values > 0])
if missing_values.sum() == 0:
    print("✅ Nenhum valor ausente encontrado!")

Colunas com valores ausentes:
Series([], dtype: int64)
✅ Nenhum valor ausente encontrado!


In [6]:
print(f"\n=== VARIÁVEIS CATEGÓRICAS E SEUS VALORES ===")

categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    unique_values = df[col].nunique()
    print(f"{col}: {unique_values} valores únicos -> {df[col].unique()}")


=== VARIÁVEIS CATEGÓRICAS E SEUS VALORES ===
gender: 3 valores únicos -> ['Male' 'Female' 'Other']
ethnicity: 5 valores únicos -> ['Asian' 'White' 'Hispanic' 'Black' 'Other']
education_level: 4 valores únicos -> ['Highschool' 'Graduate' 'Postgraduate' 'No formal']
income_level: 5 valores únicos -> ['Lower-Middle' 'Middle' 'Low' 'Upper-Middle' 'High']
employment_status: 4 valores únicos -> ['Employed' 'Unemployed' 'Retired' 'Student']
smoking_status: 3 valores únicos -> ['Never' 'Former' 'Current']
diabetes_stage: 5 valores únicos -> ['Type 2' 'No Diabetes' 'Pre-Diabetes' 'Gestational' 'Type 1']


# Detecção de Outliers

In [7]:

def detect_outliers_iqr(data, column):
    """Detecta outliers usando método IQR"""
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return len(outliers), lower_bound, upper_bound

# Analisar outliers em variáveis-chave
key_variables = ['age', 'bmi', 'glucose_fasting', 'hba1c', 'cholesterol_total', 'systolic_bp']
outlier_summary = {}

for var in key_variables:
    if var in df.columns:
        n_outliers, lower, upper = detect_outliers_iqr(df, var)
        outlier_summary[var] = {
            'count': n_outliers, 
            'percentage': (n_outliers/len(df))*100,
            'bounds': (lower, upper)
        }
        print(f"{var}: {n_outliers} outliers ({(n_outliers/len(df))*100:.2f}%) - Limites: [{lower:.2f}, {upper:.2f}]")

age: 0 outliers (0.00%) - Limites: [6.00, 94.00]
bmi: 744 outliers (0.74%) - Limites: [16.00, 35.20]
glucose_fasting: 745 outliers (0.74%) - Limites: [75.00, 147.00]
hba1c: 618 outliers (0.62%) - Limites: [4.32, 8.72]
cholesterol_total: 309 outliers (0.31%) - Limites: [98.00, 274.00]
systolic_bp: 530 outliers (0.53%) - Limites: [77.50, 153.50]
